In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [2]:

df = pd.read_csv(r"C:\Users\Saidabrorkhon\ML_Lectures\mobile_train.csv")

Data Exploration

In [4]:
df.shape

(2000, 21)

In [4]:
df.head(3)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2


Data Preprocessing

In [29]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
dtype: int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2500 non-null   int64  
 1   blue           2500 non-null   int64  
 2   clock_speed    2500 non-null   float64
 3   dual_sim       2500 non-null   int64  
 4   fc             2500 non-null   int64  
 5   four_g         2500 non-null   int64  
 6   int_memory     2500 non-null   int64  
 7   m_dep          2500 non-null   float64
 8   mobile_wt      2500 non-null   int64  
 9   n_cores        2500 non-null   int64  
 10  pc             2500 non-null   int64  
 11  px_height      2500 non-null   int64  
 12  px_width       2500 non-null   int64  
 13  ram            2500 non-null   int64  
 14  sc_h           2500 non-null   int64  
 15  sc_w           2500 non-null   int64  
 16  talk_time      2500 non-null   int64  
 17  three_g        2500 non-null   int64  
 18  touch_sc

Scaling

In [22]:
# first attemp, this way of scaling made my target variable continuous and I was forced to use Linear Regression down below.

num_col = df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df[num_col] = scaler.fit_transform(df[num_col])

In [ ]:
# scaling only features not target varaible and I used DecisionTreeClassifier down below after this.
x = df.drop(columns=['price_range'])
y = df['price_range']

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(x_scaled, columns=x.columns)

df = pd.concat([x_scaled, y], axis=1)

In [8]:
df['price_range'].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [7]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,-0.902597,-0.990050,0.830779,-1.019184,-0.762495,-1.043966,-1.380644,0.340740,1.349249,-1.101971,...,-1.408949,-1.146784,0.391703,-0.784983,0.283103,1.462493,-1.786861,-1.006018,0.986097,1
1,-0.495139,1.010051,-1.253064,0.981177,-0.992890,0.957886,1.155024,0.687548,-0.120059,-0.664768,...,0.585778,1.704465,0.467317,1.114266,-0.635317,-0.734267,0.559641,0.994018,-1.014099,2
2,-1.537686,1.010051,-1.253064,0.981177,-0.532099,0.957886,0.493546,1.381165,0.134244,0.209639,...,1.392684,1.074968,0.441498,-0.310171,-0.864922,-0.368140,0.559641,0.994018,-1.014099,2
3,-1.419319,1.010051,1.198517,-1.019184,-0.992890,-1.043966,-1.215274,1.034357,-0.261339,0.646842,...,1.286750,1.236971,0.594569,0.876859,0.512708,-0.002014,0.559641,-1.006018,-1.014099,2
4,1.325906,1.010051,-0.395011,-1.019184,2.002254,0.957886,0.658915,0.340740,0.021220,-1.101971,...,1.268718,-0.091452,-0.657666,-1.022389,-0.864922,0.730240,0.559641,0.994018,-1.014099,1


Model Selection - Linear Regression

In [5]:
lr_model = LinearRegression()
x = df.drop(columns=['price_range'])
y = df['price_range']

x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)



In [26]:
df['price_range'].nunique()

4

Model training

In [6]:
lr_model.fit(x_train, y_train)

LinearRegression()

In [ ]:
y_pred = lr_model.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

In [22]:
mse

0.09230315393522845

In [23]:
r2

0.9138303983128621

Model selection - Decision tree

In [ ]:
dt_model = DecisionTreeClassifier()
x = df.drop(columns=['price_range'])
y = df['price_range']

x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)

Model training

In [ ]:
dt = dt_model.fit(x_train, y_train)

DecisionTreeClassifier()

In [18]:
mse_tree=mean_squared_error(y_test,y_pred)
r2_tree=r2_score(y_test,y_pred)
print(mse_tree)
print(r2_tree)

0.18
0.8655688119643757


In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(dt, x, y,cv=kf, scoring='neg_mean_squared_error')
cv_scores = np.sqrt(-cv_scores)

print('Cross-Validation-Scores:', cv_scores)
print("Mean CV Score:", np.mean(cv_scores))

Cross-Validation-Scores: [0.41533119 0.3968627  0.40311289 0.42426407 0.43874822]
Mean CV Score: 0.4156638130604109
